In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [4]:
dataset = datasets.FashionMNIST(
    root="./data",
    train=True,
    download=True,
    transform=transform
)

test_dataset = datasets.FashionMNIST(
    root="./data",
    train=False,
    download=True,
    transform=transform
)

100.0%
100.0%
100.0%
100.0%


In [5]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


In [6]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [15]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 3 * 3, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))  
        x = self.pool(F.relu(self.conv2(x)))  
        x = self.pool(F.relu(self.conv3(x)))  

        x = x.view(x.size(0), -1)            
        x = F.relu(self.fc1(x))
        x = self.fc2(x)                   

        return x


In [16]:
model = CNN().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [17]:
def train(model, loader):
    model.train()
    running_loss = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(loader)


In [18]:
def evaluate(model, loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


In [22]:
epochs = 10

for epoch in range(epochs):
    train_loss = train(model, train_loader)
    val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch+1}: Loss={train_loss:.4f}, Val Acc={val_acc:.4f}")


Epoch 1: Loss=0.5090, Val Acc=0.8686
Epoch 2: Loss=0.3079, Val Acc=0.8928
Epoch 3: Loss=0.2597, Val Acc=0.9008
Epoch 4: Loss=0.2282, Val Acc=0.9061
Epoch 5: Loss=0.2032, Val Acc=0.9074
Epoch 6: Loss=0.1808, Val Acc=0.9067
Epoch 7: Loss=0.1616, Val Acc=0.9079
Epoch 8: Loss=0.1432, Val Acc=0.9143
Epoch 9: Loss=0.1267, Val Acc=0.9130
Epoch 10: Loss=0.1130, Val Acc=0.9113


In [23]:
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        y_true.extend(labels.numpy())
        y_pred.extend(predicted.cpu().numpy())


In [24]:
print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1000
           1       0.99      0.98      0.99      1000
           2       0.90      0.82      0.86      1000
           3       0.92      0.92      0.92      1000
           4       0.88      0.85      0.86      1000
           5       0.97      0.99      0.98      1000
           6       0.74      0.76      0.75      1000
           7       0.97      0.95      0.96      1000
           8       0.96      0.99      0.98      1000
           9       0.96      0.97      0.97      1000

    accuracy                           0.91     10000
   macro avg       0.91      0.91      0.91     10000
weighted avg       0.91      0.91      0.91     10000

[[886   0  10  10   4   2  76   0  12   0]
 [  3 981   0  11   3   0   1   0   1   0]
 [ 28   2 825  10  51   0  81   0   3   0]
 [ 26   3   7 917  14   0  28   0   4   1]
 [  2   2  38  32 848   2  73   0   3   0]
 [  0   0   0   0   0 989 